# Análise Exploratória no Ministério da Economia 

> Este projeto de Análise Exploratória de Dados tem como objetivo responder 05 perguntas de negócios sobre as Importações Brasileiras no primeiro semestre de 2021 e 2022, como fonte de estudos, utilizo dados extraídos do <a href="https://www.gov.br/produtividade-e-comercio-exterior/pt-br/assuntos/comercio-exterior/estatisticas/base-de-dados-bruta">Ministério da Economia.</a>

Após a conclusão deste projeto, podemos responder as seguintes perguntas de negócios:
- 01 - Qual é o valor total das Importações no período?
- 02 - Qual foi o custo das Importações por Estado?
- 03 - Qual foi o custo das Importações por Municípios?
- 04 - Quais são os principais Remetentes das Importações?
- 05 - Quais são os Produtos com Maiores Importações?

### Importando Bibliotecas e Pacotes

In [1]:
import numpy as np
import pandas as pd
import csv
import json
from sklearn.feature_extraction.text import CountVectorizer

### Acessando os dados (ref.2021)

In [2]:
#(0) Faço a importação do arquivo csv a ser analisado, (1) crio um DataFrame(df) renomeando as colunas já existentes (2)e visualizo o início.
imp_21 = "IMP_2021_MUN.csv"
dadosi21 = pd.read_csv(imp_21, names = ['Ano', 'Mês', 'Produtos', 'País(orig)', 'Estado(dest)', 'Município(dest)', 'KG_Liq', 'Total_em_Dólar'], sep = ';', encoding="utf-8-sig", low_memory=True)

/tmp/ipykernel_113390/1956096061.py:3: DtypeWarning: Columns (0,1,2,3,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  dadosi21 = pd.read_csv(imp_21, names = ['Ano', 'Mês', 'Produtos', 'País(orig)', 'Estado(dest)', 'Município(dest)', 'KG_Liq', 'Total_em_Dólar'], sep = ';', encoding="utf-8-sig", low_memory=True)


### Acessando os dados (ref.2022)

In [3]:
#(0) Faço a importação do arquivo csv a ser analisado, (1) crio um DataFrame(df) renomeando as colunas já existentes (2)e visualizo o início.
imp_22 = "IMP_2022_MUN.csv"
dadosi22 = pd.read_csv(imp_22, names = ['Ano', 'Mês', 'Produtos', 'País(orig)', 'Estado(dest)', 'Município(dest)', 'KG_Liq', 'Total_em_Dólar'], sep = ';', encoding="utf-8-sig", low_memory=True)

/tmp/ipykernel_113390/1510667406.py:3: DtypeWarning: Columns (0,1,2,3,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  dadosi22 = pd.read_csv(imp_22, names = ['Ano', 'Mês', 'Produtos', 'País(orig)', 'Estado(dest)', 'Município(dest)', 'KG_Liq', 'Total_em_Dólar'], sep = ';', encoding="utf-8-sig", low_memory=True)


### Acessando as nomeclaturas originais

In [4]:
# Realizando a leitura dos arquivos já convertidos e carregando em uma variável
with open("meses.json", "r") as jsonMes:
    meses = json.loads(jsonMes.read())
with open("produtos.json", "r") as jsonProdutos:
    produtos = json.loads(jsonProdutos.read())
with open("paises.json", "r") as jsonPais:
    paises = json.loads(jsonPais.read())
with open("estadosBr.json", "r") as jsonEstados:
    estados = json.loads(jsonEstados.read())
with open("municipios.json", "r") as jsonMun:
    municipios = json.loads(jsonMun.read())

### Limpeza e Tratamento dos Dados (ref.2021)

In [5]:
# Altero o tipo de dados das colunas para utilização com o map
dadosi21["Mês"] = dadosi21["Mês"].astype('string')
dadosi21["Produtos"] = dadosi21["Produtos"].astype('string')
dadosi21["País(orig)"] = dadosi21["País(orig)"].astype('string')
dadosi21["Estado(dest)"] = dadosi21["Estado(dest)"].astype('string')
dadosi21["Município(dest)"] = dadosi21["Município(dest)"].astype('string')

In [6]:
# Com a função drop removo a coluna "ano" - e a linha "0".
dadosi21.drop(["Ano"], axis=1, inplace=True)
dadosi21.drop([0], inplace=True)

# Por fim faço a remoção de duplicatas e valores NA
dadosi21 = dadosi21.drop_duplicates()
dadosi21 = dadosi21.dropna()

In [7]:
# Renomeio todas as colunas abaixo utilizando como valor de saída as Nomeclaturas originais
dadosi21["Mês"] = dadosi21["Mês"].map(meses,na_action='ignore')
dadosi21["Produtos"] = dadosi21["Produtos"].map(produtos,na_action='ignore')
dadosi21["País(orig)"] = dadosi21["País(orig)"].map(paises,na_action='ignore')
dadosi21["Estado(dest)"] = dadosi21["Estado(dest)"].map(estados,na_action='ignore')
dadosi21["Município(dest)"] = dadosi21["Município(dest)"].map(municipios,na_action='ignore')

In [8]:
# Seleciono o período a ser analisado com a função loc (Jan,Fev,Mar,Abril,Maio,Junho) 
dadosi21 = dadosi21.loc[dadosi21['Mês'].isin(['Janeiro','Fevereiro','Março',
                                              'Abril','Maio','Junho'])]

In [9]:
# Incluo o 'index' do pandas como propriedade do DF e reordeno o DF
dadosi21["Id_Compra"] = dadosi21.index
dadosi21 = dadosi21[['Id_Compra','Município(dest)','Estado(dest)','País(orig)','Produtos',
                     'Total_em_Dólar','KG_Liq','Mês']]

In [10]:
dadosi21

,Id_Compra,Município(dest),Estado(dest),País(orig),Produtos,Total_em_Dólar,KG_Liq,Mês
1,1,Osasco,São Paulo,México,Preparações capilares,7788,966,Março
14,14,Paranaguá,Paraná,Panamá,Bananas frescas ou secas,113,130,Janeiro
15,15,Vinhedo,São Paulo,México,"Torneiras, válvulas (incluídas as redutoras de...",4824,79,Janeiro
18,18,Nova Europa,São Paulo,Gana,Açúcares de cana ou de beterraba e sacarose qu...,259610,1080000,Abril
19,19,São Luís,Maranhão,Grécia,"Produtos de padaria, pastelaria ou da indústri...",95,125,Junho
...,...,...,...,...,...,...,...,...
1075517,1075517,Franca,São Paulo,Panamá,Outras obras de ferro ou aço,32,12,Março
1075518,1075518,Indaiatuba,São Paulo,Equador,"Bulldozers, angledozers, niveladoras, raspo-tr...",365319,68730,Junho
1075519,1075519,Santos,São Paulo,Reino Unido,"Doces, geleias, marmelades, purés e pastas de ...",76,4,Janeiro
1075520,1075520,Piracicaba,São Paulo,NaN,Outras obras de borracha vulcanizada não endur...,159,1,Fevereiro


### Limpeza e Tratamento dos Dados (ref.2022)

In [11]:
# Altero o tipo de dados das colunas para utilização com o map
dadosi22["Mês"] = dadosi22["Mês"].astype('string')
dadosi22["Produtos"] = dadosi22["Produtos"].astype('string')
dadosi22["País(orig)"] = dadosi22["País(orig)"].astype('string')
dadosi22["Estado(dest)"] = dadosi22["Estado(dest)"].astype('string')
dadosi22["Município(dest)"] = dadosi22["Município(dest)"].astype('string')

In [12]:
# Com a função drop removo a coluna "ano" e a linha "0".
dadosi22.drop(["Ano"], axis=1, inplace=True)
dadosi22.drop([0], inplace=True)

# Por fim faço a remoção de duplicatas e valores NA
dadosi22 = dadosi22.drop_duplicates()
dadosi22 = dadosi22.dropna()

In [13]:
# Renomeio todas as colunas abaixo utilizando como valor de saída as Nomeclaturas originais
dadosi22["Mês"] = dadosi22["Mês"].map(meses,na_action='ignore')
dadosi22["Produtos"] = dadosi22["Produtos"].map(produtos,na_action='ignore')
dadosi22["País(orig)"] = dadosi22["País(orig)"].map(paises,na_action='ignore')
dadosi22["Estado(dest)"] = dadosi22["Estado(dest)"].map(estados,na_action='ignore')
dadosi22["Município(dest)"] = dadosi22["Município(dest)"].map(municipios,na_action='ignore')

In [14]:
# Seleciono o período a ser analisado com a função loc (Jan,Fev,Mar,Abril,Maio,Junho) 
dadosi22 = dadosi22.loc[dadosi22['Mês'].isin(['Janeiro','Fevereiro','Março',
                                              'Abril','Maio','Junho'])]

In [15]:
# Incluo o 'index' do pandas como propriedade do DF e reordeno o DF
dadosi22["Id_Compra"] = dadosi22.index
dadosi22 = dadosi22[['Id_Compra','Município(dest)','Estado(dest)','País(orig)','Produtos',
                     'Total_em_Dólar','KG_Liq','Mês']]

In [16]:
dadosi22

,Id_Compra,Município(dest),Estado(dest),País(orig),Produtos,Total_em_Dólar,KG_Liq,Mês
1,1,Ampére,Paraná,Uruguai,Outros móveis e suas partes,47615,53482,Abril
4,4,São José,Santa Catarina,Argentina,"Camiseiros, blusas, blusas-camiseiros, de uso ...",6642,85,Janeiro
5,5,Jaraguá do Sul,Santa Catarina,Estados Unidos,"Motores e geradores, elétricos, exceto os grup...",12155328,2942192,Fevereiro
7,7,Vitória,Espírito Santo,Bahamas,"Leitelho, leite e nata coalhados, iogurte, kef...",100,32,Maio
11,11,Palhoça,Santa Catarina,Sérvia,"Fatos de saia-casaco, conjuntos, casacos, vest...",1935,11,Junho
...,...,...,...,...,...,...,...,...
1087124,1087124,Itaboraí,Rio de Janeiro,Irlanda,"Tintas e vernizes, à base de polímeros sintéti...",5496,1120,Fevereiro
1087125,1087125,Mogi Guaçu,São Paulo,NaN,"Ladrilhos e placas (lajes), para pavimentação ...",8420,19354,Janeiro
1087126,1087126,Santa Maria de Jetibá,Espírito Santo,NaN,NaN,57701,62452,Junho
1087130,1087130,Santarém,Pará,Suíça,NaN,226,30,Janeiro


> # Iniciando a Análise Exploratória

### Qual é o valor total das Importações no período?

Informações sobre o custo das Importações (Comercio Exterior -> Brasil) e o total de operações.

In [17]:
# Altero o tipo das colunas que serão utilizadas
dadosi21['Total_em_Dólar'] = dadosi21['Total_em_Dólar'].astype('int')
dadosi22['Total_em_Dólar'] = dadosi22['Total_em_Dólar'].astype('int')

In [18]:
# Calculando os valores das importações de 2021
despesa_total21 = dadosi21["Total_em_Dólar"].sum()
despesa_count21 = dadosi21["Total_em_Dólar"].count()
despesa_media21 = dadosi21["Total_em_Dólar"].mean()

In [19]:
# Calculando os valores das importações 2022
despesa_total22 = dadosi22["Total_em_Dólar"].sum()
despesa_count22 = dadosi22["Total_em_Dólar"].count()
despesa_media22 = dadosi22["Total_em_Dólar"].mean()

In [20]:
# Crio um dataframe utilizando o "pandas"
despesas21e22 = pd.DataFrame({"Ano_Base": ["2021", "2022"],
                            "Total_de_Transacoes": (despesa_count21, despesa_count22),
                            "Total_Investido_em_Importacoes": (despesa_total21, despesa_total22),
                            "Valor_medio_das_Importacoes": (despesa_media21, despesa_media22)})
# Incluo uma coluna no meu DF com passando o 'Index'
despesas21e22["Despesas21e22_id"] = despesas21e22.index

In [21]:
# Edito a formatação numérica das colunas 
despesas21e22 ["Total_de_Transacoes"] = despesas21e22["Total_de_Transacoes"].map("{:,.0f}".format)
despesas21e22 ["Total_Investido_em_Importacoes"] = despesas21e22["Total_Investido_em_Importacoes"].map("${:,.0f}".format)
despesas21e22 ["Valor_medio_das_Importacoes"] = despesas21e22["Valor_medio_das_Importacoes"].map("${:,.0f}".format)

In [22]:
# Faço uma 'Reordenação' do DF
despesas21e22 = despesas21e22[['Despesas21e22_id','Ano_Base','Total_de_Transacoes',
                               'Total_Investido_em_Importacoes','Valor_medio_das_Importacoes']]

In [23]:
despesas21e22

,Despesas21e22_id,Ano_Base,Total_de_Transacoes,Total_Investido_em_Importacoes,Valor_medio_das_Importacoes
0,0,2021,"519,419","$136,176,666,325","$262,171"
1,1,2022,"531,598","$164,070,294,634","$308,636"


## Qual foi o custo das Importações por Estado?

Faço um resumo do custo das Importações por Estado, comparando os dois periódos.

### Ano 2021

In [24]:
# Inicio agrupando os valores/operações de cada estado e realizo os devidos calculos
uf_total21 = dadosi21.groupby('Estado(dest)')['Total_em_Dólar'].sum()
uf_media21 = dadosi21.groupby("Estado(dest)")["Total_em_Dólar"].mean()
uf_operacoes21 = dadosi21.groupby("Estado(dest)")["Total_em_Dólar"].count()

In [25]:
# Crio um DataFrame para armazenar os Valores agrupados
uf_analises21 = pd.DataFrame({"Numero_de_Transacoes" : uf_operacoes21,
                              "Valor_medio_das_Importacoes" : uf_media21,
                              "Valor_total_das_Importacoes" : uf_total21})

#  Incluo uma coluna no meu DF com passando o 'Index'
uf_analises21["Uf_analises21_id"] = uf_analises21.index

In [26]:
# Edito a formatação numérica dos dados
uf_analises21 = uf_analises21.round(2)
uf_analises21 ["Valor_medio_das_Importacoes"] = uf_analises21["Valor_medio_das_Importacoes"].map("${:,.2f}".format)
uf_analises21 ["Valor_total_das_Importacoes"] = uf_analises21["Valor_total_das_Importacoes"].map("${:,.2f}".format)

In [27]:
# Faço uma 'Reordenação' do DF
uf_analises21 = uf_analises21[['Uf_analises21_id','Numero_de_Transacoes','Valor_total_das_Importacoes',
                               'Valor_medio_das_Importacoes']]

### Ano 2022

In [28]:
# Inicio agrupando os valores/operações de cada estado e realizo os devidos calculos
uf_total22 = dadosi22.groupby('Estado(dest)')['Total_em_Dólar'].sum()
uf_media22 = dadosi22.groupby("Estado(dest)")["Total_em_Dólar"].mean()
uf_operacoes22 = dadosi22.groupby("Estado(dest)")["Total_em_Dólar"].count()

In [29]:
# Crio um DataFrame para armazenar os Valores agrupados
uf_analises22 = pd.DataFrame({"Numero_de_Transacoes" : uf_operacoes22,
                              "Valor_medio_das_Importacoes" : uf_media22,
                              "Valor_total_das_Importacoes" : uf_total22})

#  Incluo uma coluna no meu DF com passando o 'Index'
uf_analises22["Uf_analises22_id"] = uf_analises22.index

In [30]:
# Edito a formatação numérica dos dados
uf_analises22 = uf_analises22.round(2)
uf_analises22 ["Valor_medio_das_Importacoes"] = uf_analises22["Valor_medio_das_Importacoes"].map("${:,.2f}".format)
uf_analises22 ["Valor_total_das_Importacoes"] = uf_analises22["Valor_total_das_Importacoes"].map("${:,.2f}".format)

In [31]:
# Faço uma 'Reordenação' do DF
uf_analises22 = uf_analises22[['Uf_analises22_id','Numero_de_Transacoes','Valor_total_das_Importacoes',
                               'Valor_medio_das_Importacoes']]

##### Unifico as duas Análises por Estados

In [32]:
# Crio um novo DF 'concatenando' os dois períodos analizados 
uf_analises2122 = pd.concat([uf_analises21,uf_analises22],
                             keys=['Ano_2021','Ano_2022'], axis=1)

In [33]:
# Incluo o 'Index' como propriedade do DF
uf_analises2122["Uf_analises2122_id"] = uf_analises2122.index

# Reordeno a concatenação 
uf_analises2122 = uf_analises2122[['Uf_analises2122_id','Ano_2021','Ano_2022']]

In [34]:
uf_analises2122.sort_values(by="Estado(dest)", ascending=False).head()

Uf_analises2122_id         Ano_2021                       \
                                  Uf_analises21_id Numero_de_Transacoes   
Estado(dest)                                                              
Tocantins               Tocantins        Tocantins                  511   
São Paulo               São Paulo        São Paulo               224443   
Sergipe                   Sergipe          Sergipe                  355   
Santa Catarina     Santa Catarina   Santa Catarina                50223   
Roraima                   Roraima          Roraima                 1400   

                                                                        \
               Valor_total_das_Importacoes Valor_medio_das_Importacoes   
Estado(dest)                                                             
Tocantins                  $997,439,221.00               $1,951,935.85   
São Paulo               $27,273,121,721.00                 $121,514.69   
Sergipe                     $26,427,825.00                  $74,444.58   
Santa Catarina           $5,929,456,041.00                 $118,062.56   
Roraima                    $137,208,957.00                  $98,006.40   

                       Ano_2022                       \
               Uf_analises22_id Numero_de_Transacoes   
Estado(dest)                                           
Tocantins             Tocantins                  537   
São Paulo             São Paulo               229222   
Sergipe                 Sergipe                  321   
Santa Catarina   Santa Catarina                52440   
Roraima                 Roraima                 1311   

                                                                        
               Valor_total_das_Importacoes Valor_medio_das_Importacoes  
Estado(dest)                                                            
Tocantins                $1,542,635,858.00               $2,872,692.47  
São Paulo               $35,632,117,825.00                 $155,448.07  
Sergipe                     $48,796,708.00                 $152,014.67  
Santa Catarina           $6,849,123,439.00                 $130,608.76  
Roraima                    $216,758,243.00                 $165,338.10

## Qual foi o custo das Importações por Municípios?

Detalhamento dos valores movimentados por Municípios no Período.

### Ano 2021

In [35]:
# Inicio agrupando os valores/operações de cada estado e realizo os devidos calculos
mun_total21 = dadosi21.groupby("Município(dest)")["Total_em_Dólar"].sum()
mun_media21 = dadosi21.groupby("Município(dest)")["Total_em_Dólar"].mean()
mun_operacoes21 = dadosi21.groupby("Município(dest)")["Total_em_Dólar"].count()

In [36]:
# Criando um DataFrame dos Municípios com os valores agrupados
mun_analises21 = pd.DataFrame({"Numero_de_Compras_no_Exterior" : mun_operacoes21,
                               "Valor_medio_das_Importacoes" : mun_media21,
                               "Valor_total_das_Importacoes" : mun_total21})


#  Incluo uma coluna no meu DF com passando o 'Index'
mun_analises21["Mun_analises21_id"] = mun_analises21.index

In [37]:
# Edito a formatação numérica dos dados
mun_analises21 ["Valor_total_das_Importacoes"] = mun_analises21["Valor_total_das_Importacoes"].map("${:,.0f}".format)
mun_analises21 ["Valor_medio_das_Importacoes"] = mun_analises21["Valor_medio_das_Importacoes"].map("${:,.0f}".format)

In [38]:
# Faço uma 'Reordenação' do DF
mun_analises21 = mun_analises21[["Mun_analises21_id","Numero_de_Compras_no_Exterior",
                                 "Valor_total_das_Importacoes", "Valor_medio_das_Importacoes"]]

In [39]:
mun_analises21.sort_values(by="Valor_total_das_Importacoes", ascending=False).head()

,Mun_analises21_id,Numero_de_Compras_no_Exterior,Valor_total_das_Importacoes,Valor_medio_das_Importacoes
Município(dest),,,,
Cuparaque,Cuparaque,2,$997,$498
Piracicaba,Piracicaba,4425,"$996,598,997","$225,220"
Manoel Urbano,Manoel Urbano,14,"$996,575","$71,184"
Pirapetinga,Pirapetinga,4,"$994,197","$248,549"
Morrinhos,Morrinhos,64,"$993,663","$15,526"


### Ano 2022

In [40]:
# Inicio agrupando os valores/operações de cada estado e realizo os devidos calculos
mun_total22 = dadosi22.groupby("Município(dest)")["Total_em_Dólar"].sum()
mun_media22 = dadosi22.groupby("Município(dest)")["Total_em_Dólar"].mean()
mun_operacoes22 = dadosi22.groupby("Município(dest)")["Total_em_Dólar"].count()

In [41]:
# Criando um DataFrame dos Municípios com os valores agrupados
mun_analises22 = pd.DataFrame({"Numero_de_Compras_no_Exterior" : mun_operacoes22,
                               "Valor_medio_das_Importacoes" : mun_media22,
                               "Valor_total_das_Importacoes" : mun_total22})


#  Incluo uma coluna no meu DF com passando o 'Index'
mun_analises22["Mun_analises22_id"] = mun_analises22.index

In [42]:
# Edito a formatação numérica dos dados
mun_analises22 ["Valor_total_das_Importacoes"] = mun_analises22["Valor_total_das_Importacoes"].map("${:,.0f}".format)
mun_analises22 ["Valor_medio_das_Importacoes"] = mun_analises22["Valor_medio_das_Importacoes"].map("${:,.0f}".format)

In [43]:
# Faço uma 'Reordenação' do DF
mun_analises22 = mun_analises22[["Mun_analises22_id","Numero_de_Compras_no_Exterior",
                                 "Valor_total_das_Importacoes", "Valor_medio_das_Importacoes"]]

In [44]:
mun_analises22.sort_values(by="Valor_total_das_Importacoes", ascending=False).head()

,Mun_analises22_id,Numero_de_Compras_no_Exterior,Valor_total_das_Importacoes,Valor_medio_das_Importacoes
Município(dest),,,,
Dois Lajeados,Dois Lajeados,32,"$998,890","$31,215"
Tarumirim,Tarumirim,1,$998,$998
Terra Roxa,Terra Roxa,10,"$996,302","$99,630"
Ladainha,Ladainha,1,$993,$993
Corupá,Corupá,79,"$991,459","$12,550"


## Quais são os principais Remetentes das Importações?

Busco identificar os principais Paises com maiores exportações para o Brasil durante o Período.

### Ano 2021

In [46]:
model_21 = dadosi21["País(orig)"]
model_21.index = range(0, 519419, 1)

In [47]:
model_21

0              México
1              Panamá
2              México
3                Gana
4              Grécia
             ...     
519414         Panamá
519415        Equador
519416    Reino Unido
519417            NaN
519418            NaN
Name: País(orig), Length: 519419, dtype: object

In [48]:
pais_21 = [str(i) for i in model_21]

In [49]:
# Criando um vetor
vetor_pais21 = CountVectorizer(analyzer = 'word', lowercase=False)

In [50]:
# Vetorizando o dataset
paises21 = vetor_pais21.fit_transform(pais_21)

In [51]:
# Retornando paises únicos
paises_unicos21 = vetor_pais21.get_feature_names_out()

In [52]:
# Criar um novo DataFrame dos Paises
dfpaises21 = pd.DataFrame(paises21.todense(), columns = paises_unicos21, index = model_21.index)

In [53]:
# Calcular o percentual 
porcent_pais21 = 100 * pd.Series(dfpaises21.sum()).sort_values(ascending = False) / dfpaises21.shape[0]

In [54]:
# Faço a 'limpeza' de 'Dados' reconhecidos com 'nan'
porcent_pais21.drop(["nan"], inplace=True)

In [55]:
# Crio um DF para melhor aproveitamento dos Dados
porcentagem_pais21 = pd.DataFrame({'Porcentagem21':porcent_pais21})

# Incluo o 'Index' como propriedade do DF
porcentagem_pais21['Pais21_id'] = porcentagem_pais21.index

# Reordeno o DF
porcentagem_pais21 = porcentagem_pais21[['Pais21_id','Porcentagem21']]

In [56]:
porcentagem_pais21.head()

,Pais21_id,Porcentagem21
Unidos,Unidos,7.451980
Estados,Estados,6.694595
Paraguai,Paraguai,6.667835
Chile,Chile,4.582620
Uruguai,Uruguai,4.473075


### Ano 2022

In [57]:
model_22 = dadosi22["País(orig)"]
model_22.index = range(0, 531598, 1)

In [58]:
model_22

0                Uruguai
1              Argentina
2         Estados Unidos
3                Bahamas
4                 Sérvia
               ...      
531593           Irlanda
531594               NaN
531595               NaN
531596             Suíça
531597               NaN
Name: País(orig), Length: 531598, dtype: object

In [59]:
pais_22 = [str(i) for i in model_22]

In [60]:
# Criando um vetor
vetor_pais22 = CountVectorizer(analyzer = 'word', lowercase=False)

In [61]:
# Vetorizando o dataset
paises22 = vetor_pais22.fit_transform(pais_22)

In [62]:
# Retornando paises únicos
paises_unicos22 = vetor_pais22.get_feature_names_out()

In [63]:
# Criar um novo DataFrame dos Paises
dfpaises22 = pd.DataFrame(paises22.todense(), columns = paises_unicos22, index = model_22.index)

In [64]:
# Calcular o percentual 
porcent_pais22 = 100 * pd.Series(dfpaises22.sum()).sort_values(ascending = False) / dfpaises22.shape[0]

In [65]:
# Faço a 'limpeza' de 'Dados' reconhecidos com 'nan'
porcent_pais22.drop(["nan"], inplace=True)

In [66]:
# Crio um DF para melhor aproveitamento dos Dados
porcentagem_pais22 = pd.DataFrame({'Porcentagem22':porcent_pais22})

# Incluo o 'Index' como propriedade do DF
porcentagem_pais22['Pais22_id'] = porcentagem_pais22.index

# Reordeno o DF
porcentagem_pais22 = porcentagem_pais22[['Pais22_id','Porcentagem22']]

In [67]:
# Um head dos 5 primeiros 
porcentagem_pais22.head()

,Pais22_id,Porcentagem22
Unidos,Unidos,7.644310
Estados,Estados,6.924029
Paraguai,Paraguai,6.349723
Chile,Chile,4.614577
Uruguai,Uruguai,4.592756


## Quais são os Produtos com Maiores Importações?

Abaixo localizo os produtos mais requisitados por importadoras no Mercado Exterior.

### Ano 2021

In [68]:
# Agrupo os Produtos e realizo a sua Contagem, em seguida, Somo os Valores de todas as suas Compras
produtos_contagem21 = dadosi21.groupby("Produtos")["Total_em_Dólar"].count()
produtos_soma21 = dadosi21.groupby("Produtos")["Total_em_Dólar"].sum()

In [69]:
# Crio um DF com os valores acima
produtosunicos21 = pd.DataFrame({"Quantidade_de_Importacoes" : produtos_contagem21,
                                 "Valor_total_das_Importacoes" : produtos_soma21})

# Incluo o 'Index no meu DF'
produtosunicos21['ProdutosUnicos21_id'] = produtosunicos21.index

In [70]:
# Faço uma pequena alteração na saída 
produtosunicos21["Valor_total_das_Importacoes"] = produtosunicos21["Valor_total_das_Importacoes"].map("${:,.0f}".format)

In [71]:
# Reordeno as 'Colunas' do meu DF
produtosunicos21 = produtosunicos21[['ProdutosUnicos21_id',
                                     'Quantidade_de_Importacoes','Valor_total_das_Importacoes']]

In [72]:
produtosunicos21.head()

,ProdutosUnicos21_id,Quantidade_de_Importacoes,Valor_total_das_Importacoes
Produtos,,,
"Abrasivos naturais ou artificiais, em pó ou em grãos, aplicados sobre matérias têxteis, papel, cartão ou outras matérias, mesmo recortados, costurados ou reunidos de outro modo","Abrasivos naturais ou artificiais, em pó ou em...",910,"$17,494,808"
"Absorventes e tampões higiênicos, cueiros e fraldas para bebês e artigos higiênicos semelhantes, de qualquer matéria","Absorventes e tampões higiênicos, cueiros e fr...",275,"$18,747,641"
"Acessórios para tubos (por exemplo, uniões, cotovelos, luvas), de alumínio","Acessórios para tubos (por exemplo, uniões, co...",355,"$1,768,908"
"Acessórios para tubos [por exemplo: uniões, cotovelos, mangas (luvas)], de cobre","Acessórios para tubos [por exemplo: uniões, co...",783,"$2,055,136"
"Acessórios para tubos [por exemplo: uniões, cotovelos, mangas (luvas)], de ferro fundido, ferro ou aço","Acessórios para tubos [por exemplo: uniões, co...",1982,"$35,711,921"


### Ano 2022

In [73]:
# Agrupo os Produtos e realizo a sua Contagem, em seguida, Somo os Valores de todas as suas Compras
produtos_contagem22 = dadosi22.groupby("Produtos")["Total_em_Dólar"].count()
produtos_soma22 = dadosi22.groupby("Produtos")["Total_em_Dólar"].sum()

In [74]:
# Crio um DF com os valores acima
produtosunicos22 = pd.DataFrame({"Quantidade_de_Importacoes" : produtos_contagem22,
                                 "Valor_total_das_Importacoes" : produtos_soma22})

# Incluo o 'Index no meu DF'
produtosunicos22['ProdutosUnicos22_id'] = produtosunicos22.index

In [75]:
# Faço uma pequena alteração na saída 
produtosunicos22["Valor_total_das_Importações"] = produtosunicos22["Valor_total_das_Importacoes"].map("${:,.0f}".format)

In [76]:
# Reordeno as 'Colunas' do meu DF
produtosunicos22 = produtosunicos22[['ProdutosUnicos22_id',
                                     'Quantidade_de_Importacoes','Valor_total_das_Importacoes']]

In [77]:
produtosunicos22.head()

,ProdutosUnicos22_id,Quantidade_de_Importacoes,Valor_total_das_Importacoes
Produtos,,,
"Abrasivos naturais ou artificiais, em pó ou em grãos, aplicados sobre matérias têxteis, papel, cartão ou outras matérias, mesmo recortados, costurados ou reunidos de outro modo","Abrasivos naturais ou artificiais, em pó ou em...",936,18927637
"Absorventes e tampões higiênicos, cueiros e fraldas para bebês e artigos higiênicos semelhantes, de qualquer matéria","Absorventes e tampões higiênicos, cueiros e fr...",272,22950485
"Acessórios para tubos (por exemplo, uniões, cotovelos, luvas), de alumínio","Acessórios para tubos (por exemplo, uniões, co...",386,2371757
"Acessórios para tubos [por exemplo: uniões, cotovelos, mangas (luvas)], de cobre","Acessórios para tubos [por exemplo: uniões, co...",867,2673341
"Acessórios para tubos [por exemplo: uniões, cotovelos, mangas (luvas)], de ferro fundido, ferro ou aço","Acessórios para tubos [por exemplo: uniões, co...",2060,56682379


#### Feito por mim, <a href="https://linkedin.com/in/mario-barcelos"> Mario Barcelos! </a> 